## Loading the libraries

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [ ]:
import tensorflow as tf
import keras
from pprint import pprint

#FilmLens veri kümesi yükleniyor


u.info yükleniyor -- u veri setindeki kullanıcı, öğe ve derecelendirme sayısı.


In [ ]:
overall_stats = pd.read_csv('ml-1m/u.info', header=None)
print("Yüklenen film lensi veri kümesine dahil olan kullanıcılar, öğeler ve derecelendirmelerin ayrıntıları: ",list(overall_stats[0]))

u.data yükleniyor -- Tam u veri seti, 1682 öğede 9436 kullanıcı tarafından 1.000.000 puan.

---



              Her kullanıcı en az 20 filmi derecelendirdi. Kullanıcılar ve öğeler
              1'den ardışık olarak numaralandırılmıştır. Veriler rastgele sıralanmıştır. Bu, sekmeyle ayrılmış bir listedir.
kullanıcı kimliği | öğe kimliği | derecelendirme | zaman damgası.
              Zaman damgaları 1/1/1970 UTC'den beri unix saniyedir

In [ ]:
## same item id is same as movie id, item id column is renamed as movie id
column_names1 = ['user id','movie id','rating','timestamp']
ratings_dataset = pd.read_csv('ml-1m/u.data', sep='\t',header=None,names=column_names1)
ratings_dataset.head() 

u.item yükleniyor -- Öğeler hakkında bilgi (filmler); bu ayrılmış bir sekme

              listesi
              film kimliği | film başlığı | çıkış tarihi | video yayın tarihi |
              IMDb URL'si | bilinmeyen | Eylem | Macera | Animasyon |
              Çocuk | komedi | Suç | belgesel | Dram | fantezi |
              Kara Film | korku | Müzikal | Gizem | romantizm | Bilim Kurgu |
              Gerilim | savaş | Batı |
              Son 19 alan türlerdir, 1 ise filmi gösterir.
              o türdense, 0 olmadığını belirtir; filmler olabilir
              aynı anda birkaç tür.
              Film kimlikleri, u.data veri setinde kullanılanlardır.


In [ ]:
d = 'movie id | movie title | release date | video release date | IMDb URL | unknown | Action | Adventure | Animation | Children | Comedy | Crime | Documentary | Drama | Fantasy | Film-Noir | Horror | Musical | Mystery | Romance | Sci-Fi | Thriller | War | Western'
column_names2 = d.split(' | ')
print(column_names2)

In [ ]:
items_dataset = pd.read_csv('ml-1m/u.item', sep='|',header=None,names=column_names2,encoding='latin-1')
items_dataset.head()

In [ ]:
movie_dataset = items_dataset[['movie id','movie title']]
movie_dataset.head()

Film kimliği sütununu kaldırdıktan sonra, items_dataset içindeki orijinal items_dataset uzunluğuna ve benzersiz satır kombinasyonlarının uzunluğuna bakılır

In [ ]:
## film kimliği sütununu kaldırdıktan sonra orijinal items_dataset uzunluğuna ve items_dataset içindeki benzersiz satır kombinasyonlarının uzunluğuna bakılır
len(items_dataset.groupby(by=column_names2[1:])),len(items_dataset)

Halihazırda eşlenmiş film başlığı için fazladan 18 film kimliği olduğunu ve kullanıcı öğesi veri kümesinde kullanıcıya aynı yinelenen film kimliğinin atandığını görebiliriz.

#Gerekli veri kümelerini birleştirme


In [ ]:
merged_dataset = pd.merge(ratings_dataset, movie_dataset, how='inner', on='movie id')
merged_dataset.head()

Benzersiz kullanıcı kimliği ve film başlığı kombinasyonu gruplandırılarak mevcut birleştirilmiş veri kümesinden bir veri kümesi oluşturulur ve bir kullanıcının aynı filme farklı durumlarda (zaman damgaları) verdiği puanların ortalaması alınır ve yeni veri kümesinde depolanır.

Bir kullanıcı tarafından belirli bir filme yönelik çoklu derecelendirme senaryosu örneği:

In [ ]:
merged_dataset[(merged_dataset['movie title'] == 'Chasing Amy (1997)') & (merged_dataset['user id'] == 894)]

## Benzersiz kullanıcı kimliği, film adı kombinasyonu ve derecelendirmeleriyle son bir rafine veri kümesi oluşturma:

In [ ]:
refined_dataset = merged_dataset.groupby(by=['user id','movie title'], as_index=False).agg({"rating":"mean"})

refined_dataset.head()

## Derin Sinir Ağları ile uğraşırken dizinin eksik değerleri olmadığından emin olmak için kullanıcıları ve film başlıklarını kodlama.

In [ ]:
user_enc = LabelEncoder()
refined_dataset['user'] = user_enc.fit_transform(refined_dataset['user id'].values)
n_users = refined_dataset['user'].nunique()

In [ ]:
item_enc = LabelEncoder()
refined_dataset['movie'] = item_enc.fit_transform(refined_dataset['movie title'].values)
n_movies = refined_dataset['movie'].nunique()

In [ ]:
refined_dataset['rating'] = refined_dataset['rating'].values.astype(np.float32)
min_rating = min(refined_dataset['rating'])
max_rating = max(refined_dataset['rating'])
n_users, n_movies, min_rating, max_rating

In [ ]:
#******************************************************************************************************************

refined_dataset.head()

## Verileri eğitim ve test olarak ayırma


In [ ]:
X = refined_dataset[['user', 'movie']].values
y = refined_dataset['rating'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=50)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

## Gömme katmanı tarafından dikkate alınması gereken faktörlerin tanımlanması


In [ ]:
n_factors = 32

Giriş dizisindeki sütunlar iki ayrı diziye bölünür. Keras bunları iki ayrı girdi olarak gördüğünden, her girdinin kendi dizisi olarak beslenmesi gerekir.

In [ ]:
X_train_array = [X_train[:, 0], X_train[:, 1]]
X_test_array = [X_test[:, 0], X_test[:, 1]]
X_train_array,X_test_array

In [ ]:
X_train, X_train_array, X_train_array[0].shape

## Etiketleri normalleştirme


In [ ]:

y_train = (y_train - min_rating)/(max_rating - min_rating)
y_test = (y_test - min_rating)/(max_rating - min_rating)
#/////////////////////////////////////////////////////////////////////////////////////////

## Softmax Derin Sinir Ağı Oluşturma


In [ ]:
## Kullanıcılar için bir giriş katmanı keras tensörü başlatılır.
## 3 boyutlu uzay başlat
# shape 1 demek 1 boyutlu vektör yığınlarının olacağını belirtir.
user = tf.keras.layers.Input(shape = (1,))

## Kullanıcıların n_faktörü için gömme katmanı
## Pozitif tam sayıları-dizileri sabit boyutlu yoğun vektörlere dönüştürür.
## Gömme matrisi için başlatıcı matrix oluşturuyo embeddings_initializer
## Gömme matrisine uygulanan düzenleyici işlevi embeddings_regularizer
## L2 düzenlileştirme cezası uygulayan bir düzenleyici. regularizers.l2
## 1e-6 0.000001 demek veri çok oluğundan 0-1 arasını ne kadar bölersek iyidir.
u = keras.layers.embeddings.Embedding(n_users, n_factors, embeddings_initializer = 'he_normal', embeddings_regularizer = tf.keras.regularizers.l2(1e-6))(user)
u = tf.keras.layers.Reshape((n_factors,))(u)

## Filmler için bir giriş katmanı keras tensörü başlatılır.
## 3 boyutlu uzay başlat
# shape 1 demek 1 boyutlu vektör yığınlarının olacağını belirtir.
movie = tf.keras.layers.Input(shape = (1,))

## filmlerin n_faktörü için gömme katmanı
## Pozitif tam sayıları-dizileri sabit boyutlu yoğun vektörlere dönüştürür.
## Gömme matrisi için başlatıcı matrix oluşturuyo embeddings_initializer
## Gömme matrisine uygulanan düzenleyici işlevi embeddings_regularizer
## L2 düzenlileştirme cezası uygulayan bir düzenleyici. regularizers.l2
## 1e-6 0.000001 demek veri çok oluğundan 0-1 arasını ne kadar bölersek iyidir.
m = keras.layers.embeddings.Embedding(n_movies, n_factors, embeddings_initializer = 'he_normal', embeddings_regularizer=tf.keras.regularizers.l2(1e-6))(movie)
## oluşan matrixi sabitler yedeniden oluşturur n_factors ile m göre matrix oluşur
m = tf.keras.layers.Reshape((n_factors,))(m)

## hem kullanıcı hem de film yerleştirmelerini istifleme
## u ile m matrixini birleştiriyoruz
x = tf.keras.layers.Concatenate()([u,m])
## matrix birleştirmesi sonucu oluşan katmanda birbiriyle az etkileşim değeri olan bağlantıyı rastgele kapatıyor. 0-1 arası değer alır. aşırı öğrenmeyi engeller.
## 0 a yakın olan değeri 0 a eşitleyerek modelin veriyle aşırı uyum sağlaması sağlanır.
## Dropout katmanı her adımda belirtilen orandaki girdiyi rassal olarak sıfıra eşitleyerek modelin veriye aşırı uyum sağlamasının (aşırı öğrenmenin) önüne geçer.
## iki noron arasındaki bağlantı az ise kapatılması  mantıklıdır. model bağlantı olduğu için onlada iletişim kuracaktır ama ilgi düzeyi az kapatılması mantıklı
x = tf.keras.layers.Dropout(0.02)(x)

## Mimariye Yoğun bir katman ekleme
## Yoğun bağlantılı noral bağlantı katmanı yap
# cıktı uzayının boyutluluğu 32 düğüm düğümler çizilir
# relu fonksiyonula çizim tahminlerinde bulunur noranlar arasındaki düğüm çiizmi
x = tf.keras.layers.Dense(128, kernel_initializer='he_normal')(x)
x = tf.keras.layers.Activation(activation='relu')(x)
x = tf.keras.layers.Dropout(0.02)(x)

x = tf.keras.layers.Dense(64, kernel_initializer='he_normal')(x)
x = tf.keras.layers.Activation(activation='relu')(x)
x = tf.keras.layers.Dropout(0.02)(x)

## cıktı katmanına ekleriz
## softmax fonksiyonunu kullanırız girdinin belirli sınıfa ait olma olasılığını 0-1 arasına koyarız
x = tf.keras.layers.Dense(32)(x)
x = tf.keras.layers.Activation(activation='softmax')(x)

## katmanları eğitim ve çıkarım özellikleriyle birlikte modelde gruplarız 
## bu artık bizim model nesnemiz
model = tf.keras.models.Model(inputs=[user,movie], outputs=x)

## model eğitim için yapılandırılır
## optimizer sgd iyileştirici fonksiyonu 
## SparseCategoricalCrossentropy Etiketler ve tahminler arasındaki çapraz entropi kaybını hesaplar. bağı koparılan noranları hesaplar
## toplamda ne kadar kayıt var. sistemdeki düzensizliği ölçer
## Düzenin düzensizliğe doğru eğiliminin ölçüsünü belirler
model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.70, patience=3, min_lr=0.000001, verbose=1)

history = model.fit(x = X_train_array, y = y_train, batch_size=64, epochs=100, verbose=1, validation_data=(X_test_array, y_test)
,shuffle=True,callbacks=[reduce_lr])


In [ ]:
plt.plot(history.history["loss"][5:])
plt.plot(history.history["val_loss"][5:])
plt.title("model loss")
plt.ylabel("loss")
plt.xlabel("epoch")
plt.legend(["train", "test"], loc="upper left")
plt.show()

## Summing up the entire code into a recommender system function:

In [ ]:
#yukarda tanımladığımız user_enc i label encoder sayesinde makine öğrenmesine algoritmalarına uygun bir hale getirdik


def recommender_system(user_id, model, n_movies):
#user id yi user_enc kullanarak normalize ediyoruz 0 ile -1 arası bi değer alıyor
  encoded_user_id = user_enc.transform([user_id])
#normalize ettiğimiz dışardan gelen kullanıcı id yi zaten elimizde normalize edilmiş id ile eşliyoruz ve onun izlediği filme ulaşıyoruz
  seen_movies = list(refined_dataset[refined_dataset['user id'] == user_id]['movie'])
#ardından kullanıcıya izlemediği filmleri önerebilmek için izlediği filmleri diğer izlemediklerinden çıkartıyoruz
  unseen_movies = [i for i in range(min(refined_dataset['movie']), max(refined_dataset['movie'])+1) if i not in seen_movies]
#eğer izlemediği film varsa if'in içine giriyoruz ve 
  model_input = [np.asarray(list(encoded_user_id)*len(unseen_movies)), np.asarray(unseen_movies)]
  predicted_ratings = model.predict(model_input)
  predicted_ratings = np.max(predicted_ratings, axis=1)
  sorted_index = np.argsort(predicted_ratings)[::-1]
  recommended_movies = item_enc.inverse_transform(sorted_index)
  print("---------------------------------------------------------------------------------")
  print(""+str(user_id)+" ID'li kullanıcı için önerilen en iyi  "+str(n_movies)+" film:")
  pprint(list(recommended_movies[:n_movies]))

In [ ]:
print("Kullanıcı ID Giriniz:")
user_id= int(input())

print("Tavsiye edilecek film sayısını girin:")
n_movies = int(input())
recommender_system(user_id,model,n_movies)
